<a href="https://colab.research.google.com/github/adithyaGHegde/Computer-Vision-Posture-Analysis/blob/main/Mini_Project_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/Basketball videos/final_final_df.csv')

df

,Unnamed: 0,video_title,timestamps,angle_elbow_right,angle_knee_right,angle_shoulder_right,angle_hip_right,angle_elbow_left,angle_knee_left,angle_shoulder_left,angle_hip_left,target
0,9,basketball_video_106_0.mp4,0.026667,49.64,163.36,21.01,173.82,39.87,158.92,17.34,161.15,0
1,10,basketball_video_106_0.mp4,33.360000,52.86,161.29,20.82,173.13,36.47,156.45,16.08,160.63,0
2,11,basketball_video_106_0.mp4,66.693333,87.35,157.94,20.36,171.34,32.79,154.16,17.23,159.74,0
3,12,basketball_video_106_0.mp4,100.026667,135.13,155.78,19.51,169.97,28.37,153.08,15.33,159.03,0
4,13,basketball_video_106_0.mp4,133.360000,106.94,152.36,20.94,167.49,28.88,150.77,15.38,158.45,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6933,58,basketball_video_066_0.mp4,1733.350000,170.65,176.98,10.96,179.77,165.81,179.36,7.40,172.86,0
6934,59,basketball_video_066_0.mp4,1766.683333,171.44,177.61,12.47,179.47,162.44,178.79,9.81,172.40,0
6935,60,basketball_video_066_0.mp4,1800.016667,172.78,177.89,13.81,179.42,163.30,179.08,11.22,172.39,0
6936,61,basketball_video_066_0.mp4,1833.350000,173.02,177.57,14.64,179.75,159.06,179.27,13.41,172.80,0


In [ ]:
# df['target'] = df['video_title'].str[-5:-4].apply(lambda x: int(x))
# df.head(5)
# df.to_csv('final_df_with_target.csv')

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, LSTM

df.drop(columns = ['Unnamed: 0', 'timestamps', 'angle_hip_right', 'angle_elbow_left', 'angle_knee_left', 'angle_shoulder_left', 'angle_hip_left'], inplace = True)
encoder = LabelEncoder()
df['video_title'] = encoder.fit_transform(df['video_title'])
df.head(5)
df.dtypes

video_title               int64
angle_elbow_right       float64
angle_knee_right        float64
angle_shoulder_right    float64
target                    int64
dtype: object

In [ ]:
X_columns = [x for x in df.columns if x!='target']
features = len(X_columns)
model = Sequential()
model.add(LSTM(100, activation='tanh', return_sequences = True, input_shape = (1, features)))
model.add(LSTM(49, activation='tanh'))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
[print(i.shape, i.dtype) for i in model.inputs]
[print(o.shape, o.dtype) for o in model.outputs]
[print(l.name, l.input_shape, l.dtype) for l in model.layers]

(None, 1, 4) <dtype: 'float32'>
(None, 1) <dtype: 'float32'>
lstm_2 (None, 1, 4) float32
lstm_3 (None, 1, 100) float32
dense_1 (None, 49) float32


[None, None, None]

In [ ]:
grouped = df.groupby('video_title')
for group in grouped:
    df_batch = pd.DataFrame(columns = ["video_title", "angle_elbow_right", "angle_knee_right", "angle_shoulder_right", "target"])
    for video in group:
        filter = df['video_title'] == video
        for key, item in df[filter].dropna().iterrows():
            df_batch = df_batch.append({'video_title': float(item['video_title']), 'angle_elbow_right': item['angle_elbow_right'], 'angle_knee_right': item['angle_knee_right'], 'angle_shoulder_right': item['angle_shoulder_right'], 'target': item['target']}, ignore_index=True)
    X = df_batch[X_columns]

    print((X.shape[0], 1, X.shape[1]))
    X = np.resize(X, (X.shape[0], 1, X.shape[1]))
    y = df_batch['target']
    model.fit(X,y,batch_size=len(X),epochs = 5)
    print('\n')

(74, 1, 4)
Epoch 1/5
1/1 [==============================] - 5s 5s/step - loss: 0.7188 - accuracy: 0.0270
Epoch 2/5
1/1 [==============================] - 0s 15ms/step - loss: 0.6335 - accuracy: 1.0000
Epoch 3/5
1/1 [==============================] - 0s 21ms/step - loss: 0.5752 - accuracy: 1.0000
Epoch 4/5
1/1 [==============================] - 0s 19ms/step - loss: 0.5220 - accuracy: 1.0000
Epoch 5/5
1/1 [==============================] - 0s 20ms/step - loss: 0.4699 - accuracy: 1.0000


(84, 1, 4)
Epoch 1/5
1/1 [==============================] - 2s 2s/step - loss: 0.4163 - accuracy: 1.0000
Epoch 2/5
1/1 [==============================] - 0s 23ms/step - loss: 0.3710 - accuracy: 1.0000
Epoch 3/5
1/1 [==============================] - 0s 19ms/step - loss: 0.3300 - accuracy: 1.0000
Epoch 4/5
1/1 [==============================] - 0s 20ms/step - loss: 0.2932 - accuracy: 1.0000
Epoch 5/5
1/1 [==============================] - 0s 20ms/step - loss: 0.2602 - accuracy: 1.0000


(73, 1, 4)
Epoch 1

KeyboardInterrupt: ignored

In [ ]:
total_acc = 0
groups = 0
for group in grouped:
    groups+=1
    df_batch = pd.DataFrame(columns = ["video_title", "angle_elbow_right", "angle_knee_right", "angle_shoulder_right", "target"])
    for video in group:
        filter = df['video_title'] == video
        for key, item in df[filter].dropna().iterrows():
            df_batch = df_batch.append({'video_title': float(item['video_title']), 'angle_elbow_right': item['angle_elbow_right'], 'angle_knee_right': item['angle_knee_right'], 'angle_shoulder_right': item['angle_shoulder_right'], 'target': item['target']}, ignore_index=True)
    X = df_batch[X_columns]

    print((X.shape[0], 1, X.shape[1]))
    X = np.resize(X, (X.shape[0], 1, X.shape[1]))
    y = df_batch['target']
    #print('\n',X)
    #model.fit(X,y,batch_size=len(X),epochs = 10)
    acc = model.evaluate(X, y, batch_size = len(X))
    total_acc += acc[1]
print(groups)
print("Accuracy =", total_acc/groups)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
from keras.callbacks import EarlyStopping


# Load and preprocess the data
data = pd.read_csv('/content/drive/MyDrive/Basketball videos/final_final_df.csv')
# Preprocess the data as necessary
y = data['target']
data.drop(columns = ['Unnamed: 0', 'video_title', 'angle_elbow_left', 'angle_knee_left', 'angle_shoulder_left', 'angle_hip_left', 'target'], inplace = True)
X = data

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Reshape the data
timesteps = 1  # Each sample is a single timestep
features = X_train.shape[1]  # Number of angles
X_train = np.resize(X_train, (X_train.shape[0], timesteps, features))
X_test = np.resize(X_test, (X_test.shape[0], timesteps, features))

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=64, dropout=0.2, recurrent_dropout=0.2, input_shape=(timesteps, features)))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# define early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Train the LSTM model
model.fit(X_train, y_train, batch_size=32, epochs=1000, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Evaluate the LSTM model
# score = model.evaluate(X_test, y_test, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

# Get prediction
y_pred = model.predict(X_test)


Epoch 1/1000
174/174 [==============================] - 7s 18ms/step - loss: 0.6915 - accuracy: 0.5450 - val_loss: 0.6730 - val_accuracy: 0.5843
Epoch 2/1000
174/174 [==============================] - 4s 23ms/step - loss: 0.6764 - accuracy: 0.5748 - val_loss: 0.6633 - val_accuracy: 0.6102
Epoch 3/1000
174/174 [==============================] - 2s 13ms/step - loss: 0.6708 - accuracy: 0.5892 - val_loss: 0.6499 - val_accuracy: 0.6398
Epoch 4/1000
174/174 [==============================] - 2s 13ms/step - loss: 0.6652 - accuracy: 0.5932 - val_loss: 0.6465 - val_accuracy: 0.6232
Epoch 5/1000
174/174 [==============================] - 2s 13ms/step - loss: 0.6618 - accuracy: 0.6068 - val_loss: 0.6367 - val_accuracy: 0.6297
Epoch 6/1000
174/174 [==============================] - 2s 13ms/step - loss: 0.6574 - accuracy: 0.6117 - val_loss: 0.6392 - val_accuracy: 0.6340
Epoch 7/1000
174/174 [==============================] - 4s 23ms/step - loss: 0.6565 - accuracy: 0.6155 - val_loss: 0.6311 - val_ac

In [ ]:
from keras.models import Model

# assuming that you have already trained the model
# and it is stored in a variable named 'model'

# save the trained model to a file
model.save('lstm_model.h5')

In [ ]:
def sigmoid(x):
    if x>=0.5:
        return 1
    else:
        return 0

y_pred = list(map(sigmoid, y_pred))
print(y_pred)

5168    1
4226    1
680     0
6480    0
3278    1
       ..
6453    0
5232    1
3132    1
4709    1
3206    1
Name: target, Length: 1388, dtype: int64
[1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

rep = classification_report(y_test, y_pred)
print(rep)

              precision    recall  f1-score   support

           0       0.69      0.51      0.59       601
           1       0.69      0.82      0.75       787

    accuracy                           0.69      1388
   macro avg       0.69      0.67      0.67      1388
weighted avg       0.69      0.69      0.68      1388

